使用python的方式从kafka读取数据太麻烦了，我们换一些简单的方式
下面的内容全部废弃：
"从kafka接收用户的行为数据
注意这里接收的过程是由spark-streaming接收到hive中，由于hive是hbase的外接表，因此相当于数据实际是保存到hbase中的
在进行这一步前，请创建对应的hive表和hbase表，创建脚本在1_init/2_init_hbase_user_list_click.ipynb中
该脚本可以全量运行"

In [79]:
class Conf:
    pass

conf = Conf()
conf.kafka_servers = "localhost:9092"
conf.kafka_topic = "user_list_click"
conf.checkpoint_path = "/resys/checkpoint"
conf.save_path = "/resys/user_list_click"
conf.save_hbase_table = "user_list_click"
conf.hive_db = "resys_one"
conf.hive_table = "user_list_click"
conf.__dict__

{'kafka_servers': 'localhost:9092',
 'kafka_topic': 'user_list_click',
 'checkpoint_path': '/resys/checkpoint',
 'save_path': '/resys/user_list_click',
 'save_hbase_table': 'user_list_click',
 'hive_db': 'resys_one',
 'hive_table': 'user_list_click'}

In [45]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

sc = SparkContext(master="yarn", appName=conf.kafka_topic)
ss = SparkSession(sc)
stream = ss.readStream.format("kafka")\
.option("kafka.bootstrap.servers", conf.kafka_servers)\
.option("subscribe", conf.kafka_topic).load()
print(stream.printSchema())
ss

21/12/10 21:54:01 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

None


In [7]:
# 输出到控制台打（测试用）
if False:
    result = stream.selectExpr("CAST(value AS STRING)").writeStream\
    .outputMode('append').format('console').start()
result.awaitTermination()

试了几种方案，最可行的是下面这种，输出到HDFS，然后使用HIVE的外接表对json进行识别
但是问题是spark以json的形式保存，会保存dataframe的完整格式，而完整格式中是带value字段的
最后以hive读出来如下面第三段代码，这样其实还是没解决问题
放弃spark-streaming的处理形式，使用kafka直接监听后解析存入hbase

In [ ]:
# 输出到HDFS
result = stream.selectExpr("CAST(value AS STRING)").writeStream\
.outputMode('append')\
.format("json")\
.option("checkpointLocation", conf.checkpoint_path)\
.option("path", conf.save_path)\
.start()
result.awaitTermination(timeout=10)

In [98]:
# 尝试对hdfs的写入路径创建外接表
from pyhive import hive
cursor = hive.connect("localhost").cursor()
cursor.execute("drop table if exists hive_test.user_list_click")
external_sql="""
 create external table if not exists hive_test.user_list_click(
    value string)
 row format serde 'org.apache.hive.hcatalog.data.JsonSerDe'
 location '/resys/user_list_click'
"""
cursor.execute(external_sql)

In [99]:
# 输出这个外接表的数据
cursor.execute("select * from hive_test.user_list_click")
for row in cursor.fetchall():
    print(row)

('{"click":"410133761","clickTime":1639145548368,"id":"2021121043905400378-410133761","list":["410133761","411006913","460151185","460176360","461045872","470182263","470306816","470349175","470383278","470526258","470802185","471025892","471100833","471166403","471325374"],"recid":"2021121043905400378","userid":"0000000001"}',)


In [77]:
stream_df = stream.selectExpr("CAST(value AS STRING)")
lines = []
def save_row(row):
    print(row)
    lines.append(row['value'])
stream_df.writeStream.foreach(save_row).start().awaitTermination(timeout=5)

False

In [ ]:
# 这种方式有问题，不再往下深入了
stream_df = stream.selectExpr("CAST(value AS STRING)")
batch_arr = []
def save_batch(batch_df, batch_id):
    batch_arr.append((batch_id, batch_df))
    batch_df.write.format("hive").mode("append").saveAsTable(conf.hive_table)
stream_df.writeStream.foreachBatch(save_batch).start().awaitTermination(timeout=10)

In [87]:
sc.stop()